# SCRAPING Subslikescript for movie scripts

In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 55.4 MB/s eta 0:00:00:00:0100:01


In [3]:
import requests
from bs4 import BeautifulSoup
import os
import time
import json
import zipfile
import os
import pandas as pd
import faiss
import numpy as np
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import torch  


2025-05-07 09:34:01.924430: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746610442.375197      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746610442.480875      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Base URL of Subslikescript
BASE_URL = "https://subslikescript.com"
MOVIE_LIST_URL = f"{BASE_URL}/movies_letter-"

In [4]:
# def get_movie_links(letter='A'):
#     """Scrapes movie page links from a letter page (A-Z)"""
#     url = f"{MOVIE_LIST_URL}{letter}"
#     res = requests.get(url)
#     soup = BeautifulSoup(res.text, 'html.parser')
#     links = soup.select('a[href^="/movie/"]')
#     movie_links = [BASE_URL + a['href'] for a in links]
#     return movie_links

In [5]:
# def get_script(url):
#     """Extracts movie title and script text from a movie page"""
#     res = requests.get(url)
#     soup = BeautifulSoup(res.text, 'html.parser')
#     title_tag = soup.find('h1')
#     script_div = soup.find('div', class_='full-script')
#     if not title_tag or not script_div:
#         return None, None
#     title = title_tag.text.strip()
#     script = script_div.get_text(separator="\n").strip()
#     return title, script


In [6]:
# def scrape_and_save_to_json(letter='A', max_movies=5, output_file='scripts.json'):
#     """Scrapes scripts and saves them as a list of dicts (RAG-ready)"""
#     links = get_movie_links(letter)
#     all_scripts = []

#     for i, link in enumerate(links[:max_movies]):
#         try:
#             title, script = get_script(link)
#             if titale and script:
#                 all_scripts.append({
#                     "title": title,
#                     "content": script,
#                     "source": link
#                 })
#                 print(f"✅ Scraped: {title}")
#             else:
#                 print(f"❌ Skipped (no script): {link}")
#         except Exception as e:
#             print(f"⚠️ Error with {link}: {e}")
#         time.sleep(1)  

#     with open(output_file, 'w', encoding='utf-8') as f:
#         json.dump(all_scripts, f, ensure_ascii=False, indent=2)
#     print(f"\n📁 Saved {len(all_scripts)} scripts to {output_file}")


In [7]:
# for letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
#     scrape_and_save_to_json(letter=letter, max_movies=10)

# RAG

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
with open('/kaggle/input/moviescripts/scripts.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [6]:
chunks = []
for movie in data:
    text = movie['content']
    sentences = sent_tokenize(text)
    chunk, temp = [], []
    for sent in sentences:
        words = sent.split()
        if len(' '.join(temp + words).split()) <= 100:
            temp.extend(words)
        else:
            chunks.append(' '.join(temp))
            temp = words[-20:]  # small overlap
    if temp:
        chunks.append(' '.join(temp))

print(f"🔹 Total chunks: {len(chunks)}")


🔹 Total chunks: 662


In [7]:
chunks = [chunk.item() if isinstance(chunk, np.ndarray) else str(chunk) for chunk in chunks]

In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks, show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/21 [00:00<?, ?it/s]

In [9]:
# Build FAISS Index
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))

In [10]:
generator = pipeline("text-generation", model="gpt2", max_length=5000)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [11]:
import pylibcugraph

In [12]:
!pip install -U bitsandbytes


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [14]:
!git config --global credential.helper store

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
from huggingface_hub import login

login()

In [17]:
model_id = "meta-llama/Llama-2-7b-chat-hf"

In [18]:
import torch
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM

In [19]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [20]:
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=1024,
    truncation=True,  
    do_sample=True,
    temperature=0.7,
    repetition_penalty=1.1,
)

Device set to use cuda:0


In [21]:
!pip install gradio

import gradio as gr

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 31.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 86.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import torch
from typing import List, Tuple, Optional
import gradio as gr

def get_relevant_context(query: str, k: int = 3) -> str:
    try:
        # Convert query to embedding
        query_embedding = model.encode([query], convert_to_tensor=True)
        # Convert to numpy and ensure correct shape
        query_np = query_embedding.cpu().numpy().astype('float32')
        
        # Ensure query is 2D for FAISS
        if len(query_np.shape) == 1:
            query_np = query_np.reshape(1, -1)
            
        # Search in FAISS
        distances, indices = index.search(query_np, k)
        
        # Safely get chunks
        relevant_chunks = []
        for idx in indices[0]:
            if 0 <= idx < len(chunks):
                relevant_chunks.append(chunks[idx])
        
        return " ".join(relevant_chunks) if relevant_chunks else ""
        
    except Exception as e:
        print(f"Error in get_relevant_context: {str(e)}")
        return ""

def generate_response(message: str, history: Optional[List[dict]] = None) -> str:
    try:
        # Input validation
        if not isinstance(message, str) or not message.strip():
            return "Please provide a valid question."
            
        # Get context
        context = get_relevant_context(message)
        
        # Create prompt with proper formatting
        prompt = f"""Context: {context}

Question: {message}

Answer: Let me help you with that."""

        # Generate response with proper error handling
        with torch.no_grad():  # Ensure no gradients are computed
            response = generator(
                prompt,
                max_length=500,
                num_return_sequences=1,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                do_sample=True,
                temperature=0.7,
                top_p=0.95,
            )

        # Extract and clean response
        generated_text = response[0]['generated_text']
        answer = generated_text.split("Answer:")[-1].strip()
        
        # Ensure we have a valid response
        if not answer:
            answer = "I apologize, but I couldn't generate a proper response. Please try rephrasing your question."
            
        return answer

    except Exception as e:
        print(f"Error in generate_response: {str(e)}")
        return "I encountered an error while processing your request. Please try again."

# Create theme and configure interface
theme = gr.themes.Soft(
    primary_hue="blue",
    secondary_hue="gray",
)

# Create Gradio interface with error handling
try:
    demo = gr.ChatInterface(
        fn=generate_response,
        title="Movie Script Chatbot",
        description="Ask me anything about movies in the database!",
        theme=theme,
        examples=[
            "What happens in the opening scene?",
            "Tell me about the main character's journey",
            "What are some memorable quotes?",
        ],
        chatbot=gr.Chatbot(type="messages")  # Updated to use new message format
    )

    # Launch with proper configuration
    demo.launch(
        share=True,
        debug=True,
        server_name="0.0.0.0",
        server_port=None,  # Let Gradio find an available port
        show_error=True
    )
    
except Exception as e:
    print(f"Error setting up Gradio interface: {str(e)}")

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:321: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'messages', will be used.
  warnings.warn(


* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://2d7ba1156c2cc86186.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Error in get_relevant_context: 'LlamaForCausalLM' object has no attribute 'encode'


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Error in get_relevant_context: 'LlamaForCausalLM' object has no attribute 'encode'


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Error in get_relevant_context: 'LlamaForCausalLM' object has no attribute 'encode'


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Error in get_relevant_context: 'LlamaForCausalLM' object has no attribute 'encode'
